<a href="https://colab.research.google.com/github/Rogerio-mack/TIC_2022_1S/blob/main/A1_solucao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="Universidade Presbiteriana Mackenzie">
</head>

<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg" width=300, align="right"> 
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->

<br>
<br>
<br>
<br>
<br>

# A1 Lab Caso: **Controle de Qualidade de Peças de uma Indústria** 
---

**pieces** é um data set para o controle de qualidade de peças produzidas por uma indústria. São empregadas quatro medidas (A, B, C e D) para o controle da qualidade das peças. As peças são então *Accept, Refurbish* ou *Reject* segundo o controle de qualidade (atributo `Quality`). A indústria ainda conta com 3 unidades para a produção dessas peças (atributo `Unit`). 

Seu trabalho é criar modelos supervisionados de classificação para predição da qualidade das peças e empregar o melhor modelo obtido (maior acuracidade) na predição da qualidade de novos casos. Em seguida você empregará um modelo de regressão linear para predição de medidas das peças.

<br>

***

<br>

Empregue os arquivos e crie os modelos conforme o final do seu TIA:

* TIA final 1,2: Arquivo [\*] = **1**, e Modelos: **Knn k=7** e **Árvore de Decisão**
* TIA final 3,4: Arquivo [\*] = **2** e Modelos: **Knn k=5** e **Árvore de Decisão**
* TIA final 5,6: Arquivo [\*] = **3** e Modelos: **Knn k=7** e **Regressão Logística**
* TIA final 7,8: Arquivo [\*] = **4** e Modelos: **Knn k=9** e  **Regressão Logística**
* TIA final 9,0: Arquivo [\*] = **5** e  Modelos: **Knn k=5** e **Árvore de Decisão com criterion='entropy'**

> Arquivo de Exemplos: http://meusite.mackenzie.br/rogerio/TIC/pieces[*].csv

> Arquivo de Novos Casos: http://meusite.mackenzie.br/rogerio/TIC/pieces[*]_new.csv

Nos modelos não empregue nenhum ou parâmetro além dos informados.


<br>

***


# Q0. Aquisição dos Dados

In [ ]:
f = input('Entre com o file number (1,2,3,4 ou 5): ')

Entre com o file number (1,2,3,4 ou 5): 1


In [ ]:
import pandas as pd
df = pd.read_csv('http://meusite.mackenzie.br/rogerio/TIC/pieces' + f + '.csv')
df.head()

,id,A,B,C,D,Quality,Unit
0,559,4.9,3.1,1.5,0.1,Reject,SP
1,629,4.8,3.4,1.6,0.2,Reject,SP
2,192,6.7,2.5,5.8,1.8,Refurbish,RJ
3,359,7.6,3.0,6.6,2.1,Refurbish,RJ
4,9,4.9,3.1,1.5,0.1,Reject,RJ


# Q1. Exploração e Preparação dos Dados

1. Explore seu data set.
2. Se houverem, elimine atributos que não devem fazer parte do modelo (treinamento).
3. Se existirem valores nulos elimine as linhas desses casos.
4. Faça o hot encode do atributo categórico `Unit` para o uso nos modelos. Em seguida exclua o atributo `Unit` original. 
5. NÃO NORMALIZE dados ou faça outras transformações não solicitadas (troca da ordem de atributos por exemplo não são recomendadas!).





In [ ]:
df = df.drop(columns=['id'])
df.head()

,A,B,C,D,Quality,Unit
0,4.9,3.1,1.5,0.1,Reject,SP
1,4.8,3.4,1.6,0.2,Reject,SP
2,6.7,2.5,5.8,1.8,Refurbish,RJ
3,7.6,3.0,6.6,2.1,Refurbish,RJ
4,4.9,3.1,1.5,0.1,Reject,RJ


In [ ]:
df.isnull().sum()

A          0
B          9
C          6
D          4
Quality    0
Unit       0
dtype: int64

In [ ]:
df = df.dropna()

In [ ]:
from sklearn.model_selection import train_test_split
seed = 1984
_, X_new, _, y_new = train_test_split(df.drop(columns=['Quality']), df['Quality'], test_size=0.01, stratify=df['Quality'], random_state=seed)

X_new.reset_index(drop=True)
X_new.to_csv('pieces' + f + '_new.csv',index=None)

X_new = pd.read_csv('/content/pieces' + f + '_new.csv')
X_new


,A,B,C,D,Unit
0,6.3,3.4,5.6,2.4,SP
1,6.7,2.5,5.8,1.8,RJ
2,5.2,4.1,1.5,0.1,BH
3,6.1,2.9,4.7,1.4,SP
4,4.8,3.4,1.9,0.2,BH


In [ ]:
hot_code = pd.get_dummies(df.Unit, prefix_sep='_')
df = pd.concat([df,hot_code],axis=1)
df = df.drop(columns=['Unit'])
df.head()

,A,B,C,D,Quality,BH,RJ,SP
0,4.9,3.1,1.5,0.1,Reject,0,0,1
1,4.8,3.4,1.6,0.2,Reject,0,0,1
2,6.7,2.5,5.8,1.8,Refurbish,0,1,0
3,7.6,3.0,6.6,2.1,Refurbish,0,1,0
4,4.9,3.1,1.5,0.1,Reject,0,1,0


In [ ]:
print('Nr de linhas e colunas após as transformações: ', df.shape)

Nr de linhas e colunas após as transformações:  (481, 8)


In [ ]:
print('Frequencias do atributo Quality:\n', df.Quality.value_counts())

Frequencias do atributo Quality:
 Refurbish    176
Reject       156
Accept       149
Name: Quality, dtype: int64


# Q2-Q3. Crie e Avalie os diferentes Modelos de Classificação

Crie os modelos conforme o final do seu TIA. Empregue o modelo de maior acurácia para predição dos novos casos. Em todos casos:

1. Empregue o `seed=1984` para a geração dos dados de treinamento e teste e para o `random_state` das Árvores de Decisão.
2. Crie os conjuntos de treinamento e teste com 0.3 para teste e dados estratificados.
3. Não empregue outros parâmetros não solicitados nos modelos.
4. Para os modelos de regressão logística ignore eventuais comentários, warnings produzidos pelo modelo.




In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 1. Variáveis preditoras e objetivo
X = df[['A', 'B', 'C', 'D', 'BH', 'RJ', 'SP']]    
y = df[['Quality']]  

# 2. Separação Treinamento e Teste
seed = 1984
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=seed)

# 3. Definição do Modelo
if f=='1': models = [ KNeighborsClassifier(7), DecisionTreeClassifier(random_state=seed) ]
if f=='2': models = [ KNeighborsClassifier(5), DecisionTreeClassifier(random_state=seed) ]
if f=='3': models = [ KNeighborsClassifier(7), LogisticRegression() ]
if f=='4': models = [ KNeighborsClassifier(9), LogisticRegression() ]
if f=='5': models = [ KNeighborsClassifier(5), DecisionTreeClassifier(criterion='entropy',random_state=seed) ]

# 4. Treinamento
for clf in models:
  clf.fit(X_train, y_train)                  

# 5. Avaliação sobre o conjunto de Teste
acc = []
for clf in models:
  y_pred = clf.predict(X_test)
  
  print(120*'*','\n')
  accuracy = accuracy_score(y_test, y_pred)
  print(clf, '\n', accuracy)

  acc.append(accuracy)
  
  cm = confusion_matrix(y_test, y_pred)
  print('\n', cm)
    
  print('\n', classification_report(y_test, y_pred))



************************************************************************************************************************ 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=7, p=2,
                     weights='uniform') 
 0.9586206896551724

 [[43  2  0]
 [ 4 49  0]
 [ 0  0 47]]

               precision    recall  f1-score   support

      Accept       0.91      0.96      0.93        45
   Refurbish       0.96      0.92      0.94        53
      Reject       1.00      1.00      1.00        47

    accuracy                           0.96       145
   macro avg       0.96      0.96      0.96       145
weighted avg       0.96      0.96      0.96       145

************************************************************************************************************************ 

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_featur

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


# Q4. Predição com o Melhor Modelo (Acurácia Geral)

## Preparação dos Dados

In [ ]:
hot_code = pd.get_dummies(X_new.Unit, prefix_sep='_')
X_new = pd.concat([X_new,hot_code],axis=1)
X_new = X_new.drop(columns=['Unit'])
X_new.head() 

,A,B,C,D,BH,RJ,SP
0,6.3,3.4,5.6,2.4,0,0,1
1,6.7,2.5,5.8,1.8,0,1,0
2,5.2,4.1,1.5,0.1,1,0,0
3,6.1,2.9,4.7,1.4,0,0,1
4,4.8,3.4,1.9,0.2,1,0,0


In [ ]:
for c in ['BH','SP','RJ']:
  if list(X_new.columns).count(c) == 0:
    X_new[c] = 0

X_new.head()     

,A,B,C,D,BH,RJ,SP
0,6.3,3.4,5.6,2.4,0,0,1
1,6.7,2.5,5.8,1.8,0,1,0
2,5.2,4.1,1.5,0.1,1,0,0
3,6.1,2.9,4.7,1.4,0,0,1
4,4.8,3.4,1.9,0.2,1,0,0


## Predição

In [ ]:
# 6. Predição de Novos casos                   
clf = models[acc.index(max(acc))]                # Escolha o Melhor modelo... 

print( clf )
print( clf.predict(X_new) )


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1984, splitter='best')
['Refurbish' 'Refurbish' 'Reject' 'Accept' 'Reject']


# Q5. Regressão Linear

Crie um modelo de Regressão Linear para o Cálculo da medida C das peças com base nas demais medidas. Empregue o modelo para estimar a medida dos C das novas peças.

In [ ]:
import statsmodels.formula.api as sm

model = sm.ols(formula='C ~ A + B + D', data=df)
result = model.fit()

y_pred = result.predict(X_new[['A','B','D']])

print(result.summary())
print(y_pred)


                            OLS Regression Results                            
Dep. Variable:                      C   R-squared:                       0.968
Model:                            OLS   Adj. R-squared:                  0.968
Method:                 Least Squares   F-statistic:                     4858.
Date:                Tue, 05 Oct 2021   Prob (F-statistic):               0.00
Time:                        13:18:27   Log-Likelihood:                -124.19
No. Observations:                 481   AIC:                             256.4
Df Residuals:                     477   BIC:                             273.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.3429      0.167     -2.052      0.0